In [33]:
import cv2 as cv
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Leer el archivo CSV con las respuestas correctas
csv_file = "train/train.csv"
df = pd.read_csv(csv_file)

# Crear listas para almacenar las características y las etiquetas
features = []
labels = []

# Procesar cada imagen y extraer las características
images_folder = 'train/train'
for filename in os.listdir(images_folder):
    if filename.endswith('.png'):
        # Leer la imagen
        img_path = os.path.join(images_folder, filename)
        img = cv.imread(img_path)
        # Extraer las características de la imagen (por ejemplo, el número de contornos)
        hsv_img = cv.cvtColor(img, cv.COLOR_BGR2HSV)
        histogram = cv.calcHist([hsv_img], [0, 1], None, [180, 256], [0, 180, 0, 256])
        most_common_color_index = np.unravel_index(np.argmax(histogram), histogram.shape)
        lower_red = np.array([0, 25, 25])
        upper_red = np.array([255, 255, 255])
        mask = cv.inRange(hsv_img, lower_red, upper_red)
        red_pixels = np.where(mask == 255)
        most_common_color_hsv = (most_common_color_index[0], most_common_color_index[1], 255)
        most_common_color_bgr = cv.cvtColor(np.uint8([[most_common_color_hsv]]), cv.COLOR_HSV2BGR)[0][0]
        img[red_pixels] = most_common_color_bgr
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        blurred = cv.GaussianBlur(gray, (5, 5), 0)
        _, thresh = cv.threshold(blurred, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
        contours, _ = cv.findContours(thresh, cv.RETR_LIST, cv.CHAIN_APPROX_NONE)
        num_contours = len(contours)
    
        # Obtener el número real de clips del archivo CSV correspondiente
        image_id = int(filename.split('-')[1].split('.')[0])
        real_clips = df.loc[df['id'] == image_id, 'clip_count'].values[0]

        # Agregar las características y la etiqueta a las listas
        features.append(num_contours)
        labels.append(real_clips)

# Convertir las listas en arrays numpy
features = np.array(features)
labels = np.array(labels)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Entrenar un modelo (ejemplo utilizando Random Forest)
#model = RandomForestRegressor()
#model.fit(X_train.reshape(-1, 1), y_train)

# Evaluar el modelo en el conjunto de prueba
#y_pred = model.predict(X_test.reshape(-1, 1))

# Calcular métricas de evaluación
#mse = mean_squared_error(y_test, y_pred)
#r2 = r2_score(y_test, y_pred)
#print(f"MSE: {mse}")
#print(f"R2 score: {r2}")
#print(f"RMSE: {np.sqrt(mse)}")

In [40]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow import keras
# Definir la arquitectura del modelo
model = keras.Sequential([
    keras.layers.Dense(256, activation='relu', input_shape=(1,)),  # Capa de entrada con 1 neurona
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1)  # Capa de salida para la regresión
])

# Compilar el modelo
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])

# Entrenar el modelo
model.fit(X_train, y_train, epochs=120, batch_size=800)

# Evaluar el modelo en el conjunto de prueba
loss, mse = model.evaluate(X_test, y_test)

# Realizar predicciones con el modelo
predictions = model.predict(X_test)

# Calcular métricas de evaluación
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

Epoch 1/120
15/15 [==============================] - 1s 4ms/step - loss: 404.8348 - mean_squared_error: 404.8348
Epoch 2/120
15/15 [==============================] - 0s 5ms/step - loss: 54.8465 - mean_squared_error: 54.8465
Epoch 3/120
15/15 [==============================] - 0s 5ms/step - loss: 18.9095 - mean_squared_error: 18.9095
Epoch 4/120
15/15 [==============================] - 0s 6ms/step - loss: 12.4814 - mean_squared_error: 12.4814
Epoch 5/120
15/15 [==============================] - 0s 5ms/step - loss: 11.7124 - mean_squared_error: 11.7124
Epoch 6/120
15/15 [==============================] - 0s 5ms/step - loss: 11.5727 - mean_squared_error: 11.5727
Epoch 7/120
15/15 [==============================] - 0s 5ms/step - loss: 11.5653 - mean_squared_error: 11.5653
Epoch 8/120
15/15 [==============================] - 0s 5ms/step - loss: 11.5467 - mean_squared_error: 11.5467
Epoch 9/120
15/15 [==============================] - 0s 5ms/step - loss: 11.5467 - mean_squared_error: 11.5467

In [41]:
#Entreno con toda la data

import cv2 as cv
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Leer el archivo CSV con las respuestas correctas
csv_file = "train/train.csv"
df = pd.read_csv(csv_file)

# Crear listas para almacenar las características y las etiquetas
features = []
labels = []

# Procesar cada imagen y extraer las características
images_folder = 'train/train'
for filename in os.listdir(images_folder):
    if filename.endswith('.png'):
        # Leer la imagen
        img_path = os.path.join(images_folder, filename)
        img = cv.imread(img_path)
        # Extraer las características de la imagen (por ejemplo, el número de contornos)
        hsv_img = cv.cvtColor(img, cv.COLOR_BGR2HSV)
        histogram = cv.calcHist([hsv_img], [0, 1], None, [180, 256], [0, 180, 0, 256])
        most_common_color_index = np.unravel_index(np.argmax(histogram), histogram.shape)
        lower_red = np.array([0, 25, 25])
        upper_red = np.array([255, 255, 255])
        mask = cv.inRange(hsv_img, lower_red, upper_red)
        red_pixels = np.where(mask == 255)
        most_common_color_hsv = (most_common_color_index[0], most_common_color_index[1], 255)
        most_common_color_bgr = cv.cvtColor(np.uint8([[most_common_color_hsv]]), cv.COLOR_HSV2BGR)[0][0]
        img[red_pixels] = most_common_color_bgr
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        blurred = cv.GaussianBlur(gray, (5, 5), 0)
        _, thresh = cv.threshold(blurred, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
        contours, _ = cv.findContours(thresh, cv.RETR_LIST, cv.CHAIN_APPROX_NONE)
        num_contours = len(contours)
    
        # Obtener el número real de clips del archivo CSV correspondiente
        image_id = int(filename.split('-')[1].split('.')[0])
        real_clips = df.loc[df['id'] == image_id, 'clip_count'].values[0]

        # Agregar las características y la etiqueta a las listas
        features.append(num_contours)
        labels.append(real_clips)

# Convertir las listas en arrays numpy
features = np.array(features)
labels = np.array(labels)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Definir la arquitectura del modelo
model = keras.Sequential([
    keras.layers.Dense(256, activation='relu', input_shape=(1,)),  # Capa de entrada con 1 neurona
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1)  # Capa de salida para la regresión
])

# Compilar el modelo
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])

# Entrenar el modelo
model.fit(features, labels, epochs=120, batch_size=800)

# Evaluar el modelo en el conjunto de prueba
loss, mse = model.evaluate(features, labels)

# Realizar predicciones con el modelo
predictions = model.predict(labels)

# Calcular métricas de evaluación
mse = mean_squared_error(labels, predictions)
r2 = r2_score(labels, predictions)

Epoch 1/120
19/19 [==============================] - 1s 4ms/step - loss: 321.2637 - mean_squared_error: 321.2637
Epoch 2/120
19/19 [==============================] - 0s 5ms/step - loss: 28.9931 - mean_squared_error: 28.9931
Epoch 3/120
19/19 [==============================] - 0s 5ms/step - loss: 13.3629 - mean_squared_error: 13.3629
Epoch 4/120
19/19 [==============================] - 0s 5ms/step - loss: 11.7400 - mean_squared_error: 11.7400
Epoch 5/120
19/19 [==============================] - 0s 5ms/step - loss: 11.5590 - mean_squared_error: 11.5590
Epoch 6/120
19/19 [==============================] - 0s 5ms/step - loss: 11.5674 - mean_squared_error: 11.5674
Epoch 7/120
19/19 [==============================] - 0s 5ms/step - loss: 11.5215 - mean_squared_error: 11.5215
Epoch 8/120
19/19 [==============================] - 0s 5ms/step - loss: 11.5240 - mean_squared_error: 11.5240
Epoch 9/120
19/19 [==============================] - 0s 5ms/step - loss: 11.5243 - mean_squared_error: 11.5243

In [36]:
#Test
features2 = []

images_folder2 = 'test/test'
for filename in os.listdir(images_folder2):
    if filename.endswith('.png'):
        # Leer la imagen
        img_path = os.path.join(images_folder2, filename)
        img = cv.imread(img_path)

        # Extraer las características de la imagen (por ejemplo, el número de contornos)
        hsv_img = cv.cvtColor(img, cv.COLOR_BGR2HSV)
        histogram = cv.calcHist([hsv_img], [0, 1], None, [180, 256], [0, 180, 0, 256])
        most_common_color_index = np.unravel_index(np.argmax(histogram), histogram.shape)
        lower_red = np.array([0, 25, 25])
        upper_red = np.array([255, 255, 255])
        mask = cv.inRange(hsv_img, lower_red, upper_red)
        red_pixels = np.where(mask == 255)
        most_common_color_hsv = (most_common_color_index[0], most_common_color_index[1], 255)
        most_common_color_bgr = cv.cvtColor(np.uint8([[most_common_color_hsv]]), cv.COLOR_HSV2BGR)[0][0]
        img[red_pixels] = most_common_color_bgr
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        blurred = cv.GaussianBlur(gray, (5, 5), 0)
        _, thresh = cv.threshold(blurred, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
        contours, _ = cv.findContours(thresh, cv.RETR_LIST, cv.CHAIN_APPROX_NONE)
        num_contours = len(contours)

        # Obtener el número real de clips del archivo CSV correspondiente
        image_id = int(filename.split('-')[1].split('.')[0])
        

        # Agregar las características y la etiqueta a las listas
        features2.append(num_contours)
        

In [42]:
features2 = np.array(features2)
y_pred2 = model.predict(features2.reshape(-1, 1))

157/157 [==============================] - 0s 2ms/step


In [43]:
ids_solucion=pd.read_csv("test/test.csv")
ids_solucion["clip_count"]=y_pred2

In [39]:
ids_solucion.to_csv("solucion4.csv",index=False)